## VEGETATION INDEX AVERAGE PLOTTING

This is only for exploring the values of the different vegetation indexes overtime.

In [10]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image
import glob
import os

In [11]:
BASE_PATH = '/home/miebakso/Desktop/Good_Crop_Bad_Crop/'

In [12]:
# x 7680,  y 1024 
def get_tile_path(x, y, band, date):
    path = f'/home/miebakso/Desktop/Good_Crop_Bad_Crop/satelite_data/phase-01/data/sentinel-2a-tile-{x}x-{y}y/timeseries/{x}-{y}-{band}-{date}.png'
    return path

In [13]:
# get the dates
paths = glob.glob(get_tile_path('7680','10240','B01','*'))
dates = []
for path in paths:
    x = len(path)
    dates.append(path[x-14:x-4])
    
# sort date  
dates.sort(key= lambda x: int(''.join(x.split('-'))))

dates[0:5]

In [14]:
# BAND 2 BLUE
# BAND 3 GREEN
# BAND 4 RED
# BAND 5 VEGETATION RED EDGE
# BAND 6 VEGETATION RED EDGE
# BAND 7 VEGETATION RED EDGE
# BAND 8 NIR
# BAND 8A VEGETATION RED EDGE
# BAND 9 WATER VAPOUR
# BAND 10 SWIR CIRRUS
# BAND 11 SWIR
# BAND 12 SWIR
BLUE = 'B02'
GREEN = 'B03'
RED = 'B04'
NIR = 'B08'

In [15]:
# get the picture
def get_image(x, y, band, date):
    path = get_tile_path(x,y,band,date)
    im = image.imread(path)
    return im

In [67]:
temp = get_tile_path('7680','10240','B02','2016-12-22')
im = Image.open(temp)

FileNotFoundError: [Errno 2] No such file or directory: '/home/miebakso/Desktop/Good_Crop_Bad_Crop/satelite_data/phase-01/data/sentinel-2a-tile-7680x-10240y/timeseries/7680-10240-TC1-2016-12-22.png'

In [17]:
temp

In [18]:
temp = get_image('7680','10240','B04','2016-12-22')
plt.imshow(temp)

In [19]:
def calulcate_NVDI(NIR, RED):
    return (NIR - RED) / (NIR + RED)

In [20]:
red = get_image('7680','10240',RED,'2016-12-22')
nir = get_image('7680','10240',NIR,'2016-12-22')
temp = calulcate_NVDI(nir,red)

In [21]:
plt.imshow(temp)

In [22]:
def calculate_ENVDI(NIR, GREEN, BLUE):
    return ((NIR+GREEN) - (2*BLUE)) / ((NIR+GREEN) + (2*BLUE)) 

In [23]:
nir = get_image('7680','10240',NIR,'2016-12-22')
green = get_image('7680','10240',GREEN,'2016-12-22')
blue = get_image('7680','10240',BLUE,'2016-12-22')
temp = calculate_ENVDI(nir,green, blue)
print(np.min(temp))
print(np.max(temp))

plt.imshow(temp)

In [24]:
def calulcate_GNDVI(NIR, GREEN):
    
    return (NIR - GREEN) / (NIR + GREEN)

In [25]:
nir = get_image('7680','10240',NIR,'2016-12-22')
green = get_image('7680','10240',GREEN,'2016-12-22')
temp = calulcate_GNDVI(nir,green)
plt.imshow(temp)

In [26]:
def calculate_SIPI(NIR, RED, BLUE):
    division = NIR - RED
    for i in range(0, len(division)):
        for j in range(0, len(division[i])):
            if division[i,j] == 0:
                division[i,j] = np.min(division[i])
    print(NIR -BLUE)
    print(division)
    result = ((NIR - BLUE) / (division))
    return result

In [27]:
nir = get_image('7680','10240',NIR,'2016-12-22')
red = get_image('7680','10240',RED,'2016-12-22')
blue = get_image('7680','10240',BLUE,'2016-12-22')
temp = calculate_SIPI(nir,red, blue)
plt.imshow(temp)
for i in range(0, len(temp)):
        for j in range(0, len(temp[i])):
            if temp[i,j] <0 or temp[i,j] >2:
                print(nir[i,j])
                print(red[i,j])
                print(blue[i,j])
                print(temp[i,j])
                print()
                


In [28]:
plt.imshow([[-409,-2,-3,-4,-1, 463]])

In [29]:
plt.imshow(nir-red)

In [30]:
temp  = Image.open(get_tile_path('7680','10240','B04','2016-12-22'))
pixels = np.array(temp.getdata()).reshape(512,512)
pixels

In [31]:
def get_wavelength(x, y, band, date):
    im = Image.open(get_tile_path(x,y,band,date))
    arr = np.array(im.getdata()).reshape(512,512)
    return arr

In [32]:
red = get_wavelength('7680','10240',RED,'2016-12-22')
nir = get_wavelength('7680','10240',NIR,'2016-12-22')
temp = calulcate_NVDI(nir,red)

In [39]:
def get_image2(x, y, band, date):
    im = Image.open(get_tile_path(x,y,band,date))
    arr = np.array(im.getdata()).reshape(512,512,3)
    return arr

In [72]:
im = Image.open(get_tile_path('7680','10240','TCI','2017-08-09'))
img = im.load()
for x in range(0,512):
    for y in range(0,512):
        a,b,c = img[x,y]
        if a>200 and b > 200 and c > 200:
            print(img[x,y])


(220, 231, 220)
(255, 231, 233)
(248, 215, 212)
(222, 230, 232)
(207, 210, 223)


In [34]:
plt.imshow(temp)

In [35]:
mask_path = '/home/miebakso/Desktop/PROJECT/Good_Crop_Bad_Crop/model/satelite_data/phase-01/data/sentinel-2a-tile-7680x-10240y/masks/sugarcane-region-mask.png'
def get_mask():
    im = Image.open(mask_path)
    pixels = im.load()
    return pixels

In [36]:
mask = get_mask()
mask[0,0]


In [37]:
# method to get the average of the tile divided by n * n filtered by mask
def get_average(image1, mask, n):
    results = []
    length = int(512/n)
    for i in range(0,n):
        for j in range(0,n):
            count = 0;
            result = 0;
            for x in range(0,length):
                for y in range(0,length):
                    x_index = (i*length) + x
                    y_index = (j* length) + y
                    if mask[x_index,y_index] == (0, 0, 0, 255):
                        pixel = image1[x_index,y_index]
                        result += pixel
                        count +=1
            if count == 0:
                results.append(0)
            else:
                results.append(result/count)
    return np.array(results)
        

In [38]:
# testing





 = get_wavelength('7680','10240',RED,'2016-12-22')
nir = get_wavelength('7680','10240',NIR,'2016-12-22')
temp = calulcate_NVDI(nir,red)
#get_average(temp,get_mask(),4)

IndentationError: unexpected indent (<ipython-input-38-361ac8233f86>, line 7)

In [ ]:
X = '7680'
Y = '10240'
n = 8
IS_IN_MASK = (0, 0, 0, 255)
nvdis = []
envdis = []
gnvdis = []
# for date in dates:
#     red = get_wavelength(X,Y,RED,date)
#     blue = get_wavelength(X,Y,BLUE,date)
#     green = get_wavelength(X,Y,GREEN,date)
#     nir = get_wavelength(X,Y,NIR,date)
    
#     nvdi = calulcate_NVDI(nir, red)
#     envdi = calculate_ENVDI(nir, green, blue)
#     gnvdi = calulcate_GNDVI(nir, green)
    
#     mask = get_mask()
#     result = get_average(nvdi,mask,n)
#     nvdis.append(result)
    
#     result = get_average(envdi,mask,n)
#     envdis.append(result)
    
#     result = get_average(gnvdi,mask,n)
#     gnvdis.append(result)
    
    

In [ ]:
nvdis

In [ ]:
xtick_loc = [i for i in range(0, len(dates))]

def plot_field_average(results, x):
    
    
    for i in range(0,len(results[0])):
        plt.figure(figsize=(12,4), dpi=100)
        result = []
        for tile in results:
            avg = tile[i]
            result.append(avg)
        plt.bar(dates, result, label = str(i))
        plt.xticks(rotation=90)
        plt.legend([str(i)])
        plt.title('Average Sub-tile Value')
        plt.grid(axis='y')
        plt.xlabel('Dates')
        plt.ylabel('VALUE')
        plt.show()
    

In [ ]:
# NVDIS
plot_field_average(nvdis,4)

In [ ]:
# ENVDIS
plot_field_average(envdis,4)

In [ ]:
# ENVDIS
plot_field_average(gnvdis,4)

In [ ]:
mask = np.ones((10,12))
mask[0,0] = 0
mask


In [ ]:
def get_partition(image, xStart, yStart, mask):
    result = [] 
    for y in range(0, len(mask)):
        for x in range(0, len(mask[0])):
            # check if the mask pixel is selected then add the img pixel to list
            if mask[y,x]==1:
                result.append(image[yStart+y, xStart+x])
    return np.array(result)

In [ ]:
temp = get_wavelength('7680','10240',RED,'2016-12-22')
res = get_partition(temp, 0,0, mask)
print(len(res))

In [ ]:
res

In [ ]:


def get_avg_vegetation_index(vi, X, Y, xStart, yStart, mask):
    result = []
    avg = []
    upper = []
    lower = []
    for date in dates[0:12]:
        red = get_wavelength(X,Y,RED,date)
        blue = get_wavelength(X,Y,BLUE,date)
        green = get_wavelength(X,Y,GREEN,date)
        nir = get_wavelength(X,Y,NIR,date)
        
        if vi == 'NVDI':
            nvdi = calulcate_NVDI(nir, red)
            res = get_partition(nvdi, xStart, yStart, mask)
            average = np.average(res)
            sd = np.std(res)

            avg.append(average)
            upper.append(average+(2*sd))
            lower.append(average-(2*sd))
            result.append(res)
    dict = {}
    dict['result'] = result
    dict['avg'] = avg
    dict['upper'] = upper
    dict['lower'] = lower
    return dict

In [ ]:
X = '7680'
Y = '10240'
xStart = 0
yStart = 0
mask = np.ones((20,20))

In [ ]:
temp = get_avg_vegetation_index('NVDI', X, Y, xStart, yStart, mask)

In [ ]:
temp

In [ ]:
xlabels = [] 
result = []
xlabel2 = []
for i in range(0, len(temp['result'])):
    xlabel2.append(i)
    for res in temp['result'][i]:
        result.append(res)
        xlabels.append(i)

In [ ]:
plt.figure(figsize=(12,4), dpi=100)

plt.scatter(xlabels, result, marker='o')

plt.xticks(xlabel2, dates[0:12])
plt.xticks(rotation=90)

# plt.title('Average Sub-tile Value')
plt.grid(axis='y')
plt.yticks(np.arange(-0.5, 1.5, step=0.1))
# plt.xlabel('Dates')
# plt.ylabel('VALUE')
plt.plot(xlabel2,temp['avg'], color='green', linestyle='dashed', label='Average')
plt.plot(xlabel2,temp['upper'], color='orange', linestyle='dashed', label='Upper')
plt.plot(xlabel2,temp['lower'], color='red', linestyle='dashed', label='Lower')
plt.show()